In [ ]:
import pandas as pd
from pathlib import Path
import os
import numpy as np
import fnmatch
from plotnine import *
import matplotlib.pyplot as plt
import seaborn as sns
from descartes import PolygonPatch
import alphashape
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import Point
import math


ProTcell_FolderLocation=r"C:\Users\data"  
dir=Path(ProTcell_FolderLocation)
os.chdir(ProTcell_FolderLocation)

# Tcell
for f in os.listdir(dir):
    if f.endswith(".txt"):
        dfNewT=pd.read_csv(f,sep=",")
        cDC1fov_FolderLocation=r"C:\Users\data\2"  
        dir=Path(cDC1fov_FolderLocation)
        for f in os.listdir(dir):
            if fnmatch.fnmatch(f, "*"+dfNewT["Sample Name"][1].split("x")[0].replace(" ","")+"*"):
            #if fnmatch.fnmatch(f, "*"+"14346-1-38"+"*"):
                dfcDC1fov=pd.read_csv(cDC1fov_FolderLocation+"\\"+f,sep=",")
                if "Cell.X.Position" in dfcDC1fov.columns:
                    dfcDC1fov = dfcDC1fov[["Cell.X.Position","Cell.Y.Position","DBSCAN Cluster"]].dropna().round(1)
                    dfcDC1fov = dfcDC1fov.rename(columns={"Cell.X.Position":"X","Cell.Y.Position":"Y"})
                elif "Cell X Position NEW" in dfcDC1fov.columns:
                    dfcDC1fov = dfcDC1fov[["Cell X Position NEW","Cell Y Position NEW","DBSCAN Cluster"]].dropna().round(1)
                    dfcDC1fov = dfcDC1fov.rename(columns={"Cell X Position NEW":"X","Cell Y Position NEW":"Y"})

In [ ]:
# import cluterType for the sample
clusterType = pd.read_csv(r"C:\Users\data\c_Type.txt")
cluster = clusterType[clusterType["Pt"].str.match(dfNewT["Sample Name"][0].split("x")[0].replace(" ",""))][["Pt","Cluster","c_Type"]]
#cluster = clusterType[clusterType["Pt"].str.match("14346-1-38 A1x9")][["Pt","Cluster","c_Type"]]

# Define hetero and homos
het = dfcDC1fov[dfcDC1fov["DBSCAN Cluster"].isin([cluster[(cluster.c_Type)=="Het"].Cluster.to_list()][0])] 
homocDC1 = dfcDC1fov[dfcDC1fov["DBSCAN Cluster"].isin([cluster[(cluster.c_Type)=="cDC1-h"].Cluster.to_list()][0])]
homoCD1a = dfcDC1fov[dfcDC1fov["DBSCAN Cluster"].isin([cluster[(cluster.c_Type)=="CD1a-h"].Cluster.to_list()][0])]

alpha = 0.01

##Create polygon for dfh141XYs:
def ifTcellsWithinCluster(typeof,a):
    dr4=[]
    for i in typeof["DBSCAN Cluster"].unique():
        Xp=typeof[(typeof["DBSCAN Cluster"]==i)].X
        Yp=typeof[(typeof["DBSCAN Cluster"]==i)].Y
        pts=list(zip(Xp, Yp))
        alpha_shape=alphashape.alphashape(pts, alpha)  

        Xmax=Xp.max()
        Xmin=Xp.min()
        Ymax=Yp.max()
        Ymin=Yp.min()

        # T cell file
        dfNewTX=dfNewT[(dfNewT.X<Xmax) & (dfNewT.X>Xmin)]
        dfNewTXy=dfNewTX[(dfNewTX.Y<Ymax) & (dfNewTX.Y>Ymin)]
        TcellPoints=list(zip(dfNewTXy.X,dfNewTXy.Y))

        # Generate Point(TcellPoint) and run alphasphere    
        Data, Booleans=[],[]
        for j in TcellPoints:
            eachPoint = Point(j)
            if eachPoint.within(alpha_shape)==True:
                Data.append(j)
                Booleans.append("Yes")
            else:
                Data.append(j)
                Booleans.append("No")
        dr4.append(pd.DataFrame({"Inside cluster?":Booleans, "Data": Data}))

    typeof=pd.concat(dr4)
    x, y = zip(*typeof["Data"])
    typeof["X"]=x
    typeof["Y"]=y
    Tcell = pd.merge(dfNewT,typeof, on=["X","Y"], how="inner")
    Tcell.to_csv(r"C:\Users\output"+"\\"+str(a)+
                 dfNewT["Sample Name"][0].split("x")[0].replace(" ","")+".txt")
    
def main():
    ifTcellsWithinCluster(het, "het")
    ifTcellsWithinCluster(homocDC1, "cDC1-h")
    ifTcellsWithinCluster(homoCD1a, "CD1a-h")

if __name__ == '__main__':
    main()

In [ ]:
# To plot (at one cluster and look at alphashape vs. points)
#s=HeteroData.Data
#fig, ax = plt.subplots()
#ax.scatter(*zip(*s))
#ax.add_patch(PolygonPatch(alpha_shape, alpha=1))
#plt.show()

In [ ]:
#clusterType["Pt"].unique()